In [ ]:
import os
import glob 
import random
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.utils import make_grid
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset
# from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, ConcatDataset
import torchvision.transforms as tt
from tqdm import tqdm_notebook
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# Custom dataset class for mammography images
class MammographyDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path= self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        label = self.labels[idx]
        return img, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
df = pd.read_csv(<your_metadata.csv>)
df = df[df['label']>0] # Including BI-RADS 1-5
df['label'] = df['label']-1 # Rescoring to 0-4
image_paths = df['name'].to_list()
labels = df['label']
labels = labels.to_list()
print(image_paths)
print(labels)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [ ]:
# Assuming image_paths and labels are already defined
train_dataset = MammographyDataset(train_images, train_labels, transform=transform)
test_dataset = MammographyDataset(test_images, test_labels, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class ResNetFC(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNetFC, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.fc_in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity() 
        self.fc1 = nn.Linear(self.fc_in_features, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, num_classes)

    def forward(self, img):
        feat = self.base_model(img)
        output = self.fc1(feat)
        output = self.fc2(output)
        output = self.fc3(output)
        return output


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
model = ResNetFC()
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Hyperparameters and other configs
config = {
    'architecture': 'feedforward',
    'lr': 0.0001,
    'scheduler_factor': 0.5,
    'scheduler_patience': 2,
    'scheduler_min_lr': 1e-6,
    'epochs': 40
}

optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    'min',
    factor=config['scheduler_factor'],
    patience=config['scheduler_patience'],
    min_lr=config['scheduler_min_lr']
)

In [ ]:
# Training loop
min_loss = 1000
for epoch in tqdm_notebook(range(config['epochs'])):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{config['epochs']}], Train Loss: {running_loss / len(train_dataloader):.6f}, learning rate : {optimizer.param_groups[0]['lr']}")
    
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
        scheduler.step(test_loss)
        
    print(f"Test Loss: {test_loss / len(test_dataloader):.6f}")
        
    if test_loss < min_loss:
        state_dict = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'train_loss': running_loss,
            'val_loss': test_loss,
            'best_val_loss': min_loss,
        }
        weight_path = f"<your_dir>/resnet50_pretrained_withhiddenlayers_{epoch}.pth"
        torch.save(model.state_dict(), weight_path)
        min_loss = test_loss


In [ ]:
model = ResNetFC()
weight_path = '<your_best_weight_path>'
model.load_state_dict(torch.load(weight_path))
model = model.to(device)

In [ ]:
model.eval()
test_loss = 0.0
cm_labels = []
cm_preds = []
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        cm_preds += list(preds.cpu().numpy())
        cm_labels += list(labels.cpu().numpy())
        loss = criterion(outputs, labels)
        test_loss += loss.item()
print(f"Test Loss: {test_loss / len(test_dataloader):.6f}")

In [ ]:
def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

In [ ]:
def grad_plot(image_tensor, single_label):
    input_tensor = image_tensor.unsqueeze(0).cuda()  # Add batch dimension if not already present
    target_layers = [model.layer4[-1]] # Define the target layers and create the GradCAM object
    cam = GradCAM(model=model, target_layers=target_layers)
    targets = [ClassifierOutputTarget(single_label)] # Define the target class for Grad-CAM
    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :] # Generate the CAM
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    denormalized_tensor = denormalize(image_tensor.clone(), mean, std)
    rgb_img = denormalized_tensor.permute(1, 2, 0).cpu().numpy()  # Convert CHW to HWC format
    rgb_img = (rgb_img * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8 for visualization
    visualization = show_cam_on_image(rgb_img / 255.0, grayscale_cam, use_rgb=True)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_label = predicted.item()

    plt.figure(figsize=(6, 3))

    plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
    plt.imshow(rgb_img, cmap='gray')
    plt.title(f"Original : BIRADS {single_label+1}")
    plt.axis('off')

    plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
    plt.imshow(visualization)
    plt.title(f"Grad-CAM : BIRADS {predicted_label+1}")
    plt.axis('off')

    plt.show()

In [ ]:
# Plot up to 40 images, but feel free to adjust the number as needed.
count = 0
for img, label in test_dataset:
    grad_plot(img, label)
    count +=1
    if count >=40:
        break

In [ ]:
# Plot the confusion matrix using seaborn
cm = confusion_matrix(cm_labels, cm_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['1', '2', '3', '4', '5'],
            yticklabels=['1', '2', '3', '4', '5'])

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import re

# The input text
text = """ Your model output, for example :
Epoch [1/40], Train Loss: 1.367641, learning rate : 0.0001
Test Loss: 1.330273
Epoch [2/40], Train Loss: 0.992598, learning rate : 0.0001
Test Loss: 1.575209
Epoch [3/40], Train Loss: 0.595229, learning rate : 0.0001
Test Loss: 1.421532
"""

# Regular expressions to extract epochs, train losses, and validation losses
epoch_pattern = re.compile(r"Epoch \[(\d+)/\d+\]")
train_loss_pattern = re.compile(r"Train Loss: ([\d\.]+)")
val_loss_pattern = re.compile(r"Test Loss: ([\d\.]+)")

# Extracting the data
epochs = [int(epoch) for epoch in epoch_pattern.findall(text)]
train_losses = [float(train_loss) for train_loss in train_loss_pattern.findall(text)]
val_losses = [float(val_loss) for val_loss in val_loss_pattern.findall(text)]

# Output the extracted data
print("Epochs:", epochs)
print("Train Losses:", train_losses)
print("Validation Losses:", val_losses)

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Validation Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def calculate_accuracy(all_preds, all_labels):
    total_samples = len(all_labels)
    correct_predictions = sum(1 for pred, label in zip(all_preds, all_labels) if pred == label)
    accuracy = correct_predictions / total_samples
    return accuracy

accuracy = calculate_accuracy(cm_preds, cm_labels)
print("Accuracy:", accuracy)
